In [1]:
import langchain
langchain.__version__

'0.3.22'

In [2]:
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [6]:
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun

search_tool = Tool(
    name = "DuckDuckGo search",
    func= DuckDuckGoSearchRun().run,
    description="Use this tool to perform web searches using DuckDuckGo"
)

llm = OpenAI(temperature = 0.5)

tool = [search_tool]

agent = initialize_agent(tool , llm , agent="zero-shot-react-description", verbose = True)

response = agent.run("What are the latest top raning movies in 2025?")

print(response)

C:\Users\Darshana\AppData\Local\Temp\ipykernel_20492\3732921092.py:15: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run("What are the latest top raning movies in 2025?")




> Entering new AgentExecutor chain...
 I should use DuckDuckGo search to find a list of top ranking movies in 2025
Action: DuckDuckGo search
Action Input: 'top ranking movies 2025'
Observation: For example, a movie released over Thanksgiving in 2024 will most likely earn money in 2024 and 2025. Click on the individual movie to see its total gross over all the years in which it played. Inflation-adjusted figures are based on ticket sale estimates. See Top 2025 Movies at the Domestic Box Office for the total domestic box office for the ... This film could end up being a stinker, leaving fans far less satisfied than 2023's The Super Mario Bros. Movie, but even poor reviews may not matter on its journey to half a billion dollars. 2025 Box Office Honorable Mentions. While these honorable mentions didn't crack the Top 10, they each carry unique potential. Outside of a few box-office smashes, 2024 was a relatively quiet year for movies, full of fascinating breakouts and leftfield successes,

In [7]:
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper

wiki_search_tool = Tool(
    name = "Wikipedia search",
    func= WikipediaAPIWrapper().run,
    description="Use this tool to perform search for wikipedia"
)

llm = OpenAI(temperature = 0.5)

tool = [wiki_search_tool]

agent = initialize_agent(tool , llm , agent="zero-shot-react-description", verbose = True)

response = agent.run("What are the latest top raning movies in 2025?")

print(response)



> Entering new AgentExecutor chain...
 I should use Wikipedia to search for the latest top ranking movies in 2025
Action: Wikipedia search
Action Input: "latest top ranking movies in 2025"
Observation: Page: List of highest-grossing films
Summary: Films generate income from several revenue streams, including theatrical exhibition, home video, television broadcast rights, and merchandising. However, theatrical box-office earnings are the primary metric for trade publications in assessing the success of a film, mostly because of the availability of the data compared to sales figures for home video and broadcast rights, but also because of historical practice. Included on the list are charts of the top box-office earners (ranked by both the nominal and real value of their revenue), a chart of high-grossing films by calendar year, a timeline showing the transition of the highest-grossing film record, and a chart of the highest-grossing film franchises and series. All charts are ranked by

In [19]:
import sqlite3

connection = sqlite3.connect("example_new.db")
cursor = connection.cursor()

cursor.execute(""" 
               CREATE  TABLE IF NOT EXISTS inventory (
                   id INTEGER PRIMARY KEY,
                   product_name TEXT NOT NULL,
                   stock_quanity INTEGER NOT NULL)""")

cursor.executemany("""
        INSERT INTO inventory (product_name, stock_quanity) VALUES (? ,?)
        """,[ ("Product A" , 150),
             ("Product B" , 200),
            ("Product C" , 250),
            ("Product E" , 300),
            ("Product F" , 400),
            ("Product G" , 150)
        ])

connection.commit()
connection.close()

print("Database created and populated with sample data.")

Database created and populated with sample data.


In [16]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit



In [21]:
db = SQLDatabase.from_uri("sqlite:///example_new.db")

llm = OpenAI(temperature = 0.5)

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

agent = initialize_agent(
    tools=tools,
    llm = llm,
    agent="zero-shot-react-description",
    verbose = True
)

query = "How many units of product A are in the stock?"
response1 = agent.run(query)
print("Response:" , response1)



> Entering new AgentExecutor chain...
 We need to query the database to get the stock information for product A.
Action: sql_db_query
Action Input: SELECT units FROM stock WHERE product = 'A'
Observation: Error: (sqlite3.OperationalError) no such table: stock
[SQL: SELECT units FROM stock WHERE product = 'A']
(Background on this error at: https://sqlalche.me/e/20/e3q8)
Thought: Since we got an error, we need to check the schema of the database to see if the table 'stock' exists.
Action: sql_db_schema
Action Input: stock
Observation: Error: table_names {'stock'} not found in database
Thought: We need to get a list of tables in the database to see if 'stock' is listed.
Action: sql_db_list_tables
Action Input: 
Observation: inventory
Thought: Since 'stock' is not listed, we need to use sql_db_query_checker to double check our query before executing it.
Action: sql_db_query_checker
Action Input: SELECT units FROM inventory WHERE product = 'A'
Observation: 
SELECT units 
FROM inventory 
W

ArxivQueryRun

FILE AGENTS

In [22]:
from langchain_community.tools.file_management import ReadFileTool
from langchain.agents import initialize_agent , Tool
from langchain.llms import OpenAI 

In [25]:
read_file_tool = Tool(
    name="Read File",
    func=ReadFileTool().run,
    description="Use this tool to read the content of a text file in the current working directory. Provide the file name as input."
)

llm = OpenAI(temperature=0.5)

tools = [read_file_tool]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)

file_name ="example.txt"

response = agent.run(f"Read the content of the file named {file_name}.")

print(response)




> Entering new AgentExecutor chain...
 I should use the Read File tool to read the content of the file.
Action: Read File
Action Input: example.txt
Observation: At NASA, “systems engineering” is defined as a methodical, multi-disciplinary approach for the design, realization, technical management, operations, and retirement of a system. A “system” is the combination of elements that function together to produce the capability required to meet a need. The elements include all hardware, software, equipment, facilities, personnel, processes, and procedures needed for this purpose; that is, all things required to produce system-level results. The results include system-level qualities, properties, characteristics, functions, behavior, and performance. The value added by the system as a whole, beyond that contributed independently by the parts, is primarily created by the relationship among the parts; that is, how they are interconnected. It is a way of looking at the “big picture” when m

In [24]:
%pwd

'c:\\Darshana\\AgenticAI\\LangChain_AI_Agents\\LangChain-AI-Agents'